In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
root_path = "/content/drive/MyDrive/DFS/pu/thesis/"

place_path = f"{root_path}/crawl"
data_path = f"{root_path}/the_new_one/data"

!ls $data_path

df_aspect.csv		 df_preprocess.csv		  filtered_data.csv
df_aspect_stopwords.csv  df_preprocess_stopwords.csv	  image
df_none.csv		 df_with_aspect_sentiment.csv	  old
df_none_none.csv	 df_with_aspect_sentiment.gsheet  short_data.csv


In [3]:
import pickle

with open(f"{place_path}/places.pkl", "rb") as f:
    old_places = pickle.load(f)

len(old_places)

7259

In [4]:
places = {}

for id, place in old_places.items():
    cid = place['googleMapsUri'].split("cid=")[-1].split("&")[0]
    places[cid] = place

len(places)

7259

f_out: crawl data

f_in: input url

In [5]:
import json
import numpy as np

with open(f"{data_path}/image/data.out", "r", encoding="utf-8") as f_out:
    rows_out = f_out.readlines()

with open(f"{data_path}/image/links.in", "r", encoding="utf-8") as f_in:
    rows_in = f_in.readlines()

data = {}
for link, row in zip(rows_in, rows_out):
    link = link.strip().strip("\n")
    cid = link.split("cid=")[-1].split("&")[0]

    data[cid] = None

    row = row.strip().strip("\n")
    if row == "null":
        continue

    res_data = json.loads(row)

    res_data["total_reviews"] = len(res_data["reviews"])
    res_data["link"] = link

    data[cid] = res_data

print("Total:", len(data))
print()

data = {k: v for k, v in data.items() if v is not None and v["total_reviews"] > 0}

def desc_reviews(data):
    total_reviews = [entry['total_reviews'] for cid, entry in data.items()]
    print("Total restaurants:", len(data))
    print("Total reviews:", np.sum(total_reviews))

    print("Min:", np.min(total_reviews))
    print("Max:", np.max(total_reviews))
    print("Mean:", np.mean(total_reviews))
    print("Median:", np.median(total_reviews))
    print("Std:", np.std(total_reviews))

    return total_reviews

print("Filtered:", len(data))
total_reviews = desc_reviews(data)

Total: 7259

Filtered: 2903
Total restaurants: 2903
Total reviews: 27604
Min: 1
Max: 254
Mean: 9.50878401653462
Median: 2.0
Std: 22.682304094863685


In [73]:
import pandas as pd
from datetime import datetime

df = []

for cid, entry in data.items():
    place = places[cid]

    for review in entry["reviews"]:

        temp = {
            "place_id": place['id'],
            "link": entry["link"],
            "name": place['displayName']['text'],
            "status": place['businessStatus'],
            "location": place.get('shortFormattedAddress', place['formattedAddress']),
            "overall_rating": entry["star"],
            "price": entry["price"].split(r"\x")[0].strip("₫").strip().strip("N").strip() if entry["price"] is not None else None,
            "rating": review["star"],
            "review_time": review["time"],
            "comment": review["comment"]
        }
        if review["aspects"] is not None:
          for aspect, value in review["aspects"].items():
              temp[aspect] = value

        for component in place["addressComponents"]:
            temp[component["types"][0]] = component["shortText"]

        df.append(temp)

df = pd.DataFrame(df)
df['review_time'] = pd.to_datetime(df['review_time'], format='%m,%Y')

df = df[df['administrative_area_level_1'] == 'Hồ Chí Minh']

df.drop(columns=["country", "administrative_area_level_1", "postal_code"], inplace=True)
df.rename(columns = {'administrative_area_level_2': 'area'}, inplace=True)

df.reset_index(drop=True, inplace=True)
df

,place_id,link,name,status,location,overall_rating,price,rating,review_time,comment,...,route,area,plus_code,neighborhood,subpremise,sublocality_level_1,premise,administrative_area_level_3,point_of_interest,locality
0,ChIJp1UGz2YpdTERL109bi26Azs,https://maps.google.com/?cid=42524471774536901...,Cơm niêu KOMBO - L5-08 TTTM Parcmall,OPERATIONAL,"L5 08 TTTM Parcmall, Phường 15",3.7,Vừa,4.0,2024-09-01,"Phần cơm trung bình 90k/phần, nước trung bình ...",...,08 TTTM Parcmall,Tân Bình,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ChIJp1UGz2YpdTERL109bi26Azs,https://maps.google.com/?cid=42524471774536901...,Cơm niêu KOMBO - L5-08 TTTM Parcmall,OPERATIONAL,"L5 08 TTTM Parcmall, Phường 15",3.7,Vừa,5.0,2024-09-01,"Cơm lạ miệng, có lớp cơm cháy giòn giòn ăn ngo...",...,08 TTTM Parcmall,Tân Bình,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ChIJTYxwqQQpdTERu-toZpUuJhY,https://maps.google.com/?cid=15960143371525764...,Trịnh trần phương tuấn,OPERATIONAL,"RJFH+6VM, Phường 15",4.2,None,5.0,2019-09-01,Tuyệt vời,...,NaN,Tân Bình,RJFH+6VM,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ChIJNd_JkGIpdTERXRtR4acBq-w,https://maps.google.com/?cid=17053726234519870...,Domino's Pizza Đường Phan Huy Ích,OPERATIONAL,"7 Phan Huy Ích, Phường 15",4.4,None,5.0,2024-11-01,🌟🌟🌟🌟🌟\n\nMình rất hài lòng khi ghé Domino's Pi...,...,Phan Huy Ích,Tân Bình,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ChIJNd_JkGIpdTERXRtR4acBq-w,https://maps.google.com/?cid=17053726234519870...,Domino's Pizza Đường Phan Huy Ích,OPERATIONAL,"7 Phan Huy Ích, Phường 15",4.4,None,4.0,2024-11-01,"Cỡ 9 có thể 2ng ăn ,sức ăn mạnh chắc 1 người ăn.",...,Phan Huy Ích,Tân Bình,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27597,ChIJmeUOGdIrdTERWnUI3xPk4es,https://maps.google.com/?cid=16997117242670347...,Nhà hàng Hoa - Gió,OPERATIONAL,"527 Trần Văn Mười, Ấp 3, Xuân Thới Thượng",4.1,None,5.0,2020-12-01,"Ngon, nhân viên nhiệt tình.",...,Trần Văn Mười,Hóc Môn,NaN,Ấp 3,NaN,NaN,NaN,NaN,NaN,NaN
27598,ChIJyZIjHQArdTERFfwWcYVQrak,https://maps.google.com/?cid=12226517097438968...,Cơm Gà quay Hồng Kông Hương Xưa,OPERATIONAL,"108 Trần Văn Mười, Ấp 3, Xuân Thới Thượng",5.0,None,5.0,2024-08-01,"Cơm dẻo, thịt gà quay rất ngon",...,Trần Văn Mười,Hóc Môn,NaN,Ấp 3,NaN,NaN,NaN,NaN,NaN,NaN
27599,ChIJyZIjHQArdTERFfwWcYVQrak,https://maps.google.com/?cid=12226517097438968...,Cơm Gà quay Hồng Kông Hương Xưa,OPERATIONAL,"108 Trần Văn Mười, Ấp 3, Xuân Thới Thượng",5.0,None,5.0,2024-05-01,"Cơm dẻo ngon, thịt giòn ngon, nước chấm ok! Th...",...,Trần Văn Mười,Hóc Môn,NaN,Ấp 3,NaN,NaN,NaN,NaN,NaN,NaN
27600,ChIJhZgI3rUrdTER3vtylCA4fsU,https://maps.google.com/?cid=14230873585118018...,Mr Tô,OPERATIONAL,"Hẻm kế bên 10/1G XTT 6-2-5, Tổ 3, Ấp 3",5.0,None,5.0,2022-10-01,"Trà sữa ngon, đồ ăn okie. Anh chị chủ tư vấn n...",...,Ấp 3,Hóc Môn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
import os

old_df = pd.read_csv(os.path.join(data_path, "old", "preprocessed_data.csv"))
old_df['review_time'] = pd.to_datetime(old_df['review_time'])

old_df['location'] = old_df.apply(
    lambda row: f"{row['name']}, {row['street_number']}, {row['route']}, {row['area']}" if str(row['street_number']) != "nan" else f"{row['name']}, {row['route']}, {row['area']}", axis=1
)

old_df

,place_id,name,status,overall_rating,price,rating,review_time,comment,Đồ ăn,Dịch vụ,Bầu không khí,street_number,route,area,Loại hình bữa ăn,location
0,ChIJNd_JkGIpdTERXRtR4acBq-w,Domino's Pizza Đường Phan Huy Ích,OPERATIONAL,4.4,100-200,1.0,2024-01-01,Ngồi chờ 30’ xong nhân viên ra báo bánh bị hư ...,2.0,1,1.0,7,Phan Huy Ích,Tân Bình,NaN,"Domino's Pizza Đường Phan Huy Ích, 7, Phan Huy..."
1,ChIJNd_JkGIpdTERXRtR4acBq-w,Domino's Pizza Đường Phan Huy Ích,OPERATIONAL,4.4,100-200,5.0,2023-07-01,"Pizza ngon , phục vụ tận tình , không gian tho...",5.0,5,5.0,7,Phan Huy Ích,Tân Bình,NaN,"Domino's Pizza Đường Phan Huy Ích, 7, Phan Huy..."
2,ChIJNd_JkGIpdTERXRtR4acBq-w,Domino's Pizza Đường Phan Huy Ích,OPERATIONAL,4.4,100-200,1.0,2023-08-01,Đây là cái bánh pizza phô mai bên phan huy ích...,1.0,4,4.0,7,Phan Huy Ích,Tân Bình,NaN,"Domino's Pizza Đường Phan Huy Ích, 7, Phan Huy..."
3,ChIJNd_JkGIpdTERXRtR4acBq-w,Domino's Pizza Đường Phan Huy Ích,OPERATIONAL,4.4,100-200,4.0,2023-01-01,"Ngon nhưng web Order hay bị lag, nên khó đặt q...",5.0,4,3.0,7,Phan Huy Ích,Tân Bình,NaN,"Domino's Pizza Đường Phan Huy Ích, 7, Phan Huy..."
4,ChIJNd_JkGIpdTERXRtR4acBq-w,Domino's Pizza Đường Phan Huy Ích,OPERATIONAL,4.4,100-200,4.0,2023-01-01,Pizza ăn ngon. Không gian rộng rãi thoáng mát..,4.0,4,4.0,7,Phan Huy Ích,Tân Bình,Bữa trưa,"Domino's Pizza Đường Phan Huy Ích, 7, Phan Huy..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14497,ChIJd_2LSKMrdTERlUNKSqVWMFs,Bún bò Vị Nguyên,OPERATIONAL,4.7,1-100.000,5.0,2024-01-01,"Thoáng mát hợp khẩu vị, sẽ ghé lại.",5.0,Ăn tại chỗ,NaN,NaN,QL22,Hóc Môn,Bữa trưa,"Bún bò Vị Nguyên, QL22, Hóc Môn"
14498,ChIJd_2LSKMrdTERlUNKSqVWMFs,Bún bò Vị Nguyên,OPERATIONAL,4.7,1-100.000,5.0,2024-03-01,Quá ngon sài gòn tôi ơi,5.0,5,5.0,NaN,QL22,Hóc Môn,NaN,"Bún bò Vị Nguyên, QL22, Hóc Môn"
14499,ChIJd_2LSKMrdTERlUNKSqVWMFs,Bún bò Vị Nguyên,OPERATIONAL,4.7,1-100.000,1.0,2024-05-01,Không như quảng cáo,1.0,1,1.0,NaN,QL22,Hóc Môn,Bữa tối,"Bún bò Vị Nguyên, QL22, Hóc Môn"
14500,ChIJd_2LSKMrdTERlUNKSqVWMFs,Bún bò Vị Nguyên,OPERATIONAL,4.7,1-100.000,1.0,2024-11-01,"Lần đầu ăn ok, thịt nhiều bún cũng nhiều, nhữn...",1.0,1,1.0,NaN,QL22,Hóc Môn,Bữa sáng,"Bún bò Vị Nguyên, QL22, Hóc Môn"


In [75]:
for idx, row in old_df.iterrows():
    res_df = df[df['place_id'] == row['place_id']]
    comment_row = res_df[res_df['comment'] == row['comment']]

    if len(comment_row) == 0:
        temp_idx = len(df)

        for col in df.columns:
            if col not in row.keys():
                df.at[temp_idx, col] = None
            else:
                df.at[temp_idx, col] = row[col]
        continue

    comment_idx = comment_row.index[0]

    for col in row.keys():
        if col not in df.columns and col != "link":
            df[col] = None

        if df.at[comment_idx, col] is not None:
            continue

        df.at[comment_idx, col] = row[col]

    df.at[comment_idx, 'review_time'] = row['review_time']

In [76]:
df.drop_duplicates(subset=['comment'], inplace=True)
df.reset_index(drop=True, inplace=True)

df

,place_id,link,name,status,location,overall_rating,price,rating,review_time,comment,...,subpremise,sublocality_level_1,premise,administrative_area_level_3,point_of_interest,locality,Đồ ăn,Dịch vụ,Bầu không khí,Loại hình bữa ăn
0,ChIJp1UGz2YpdTERL109bi26Azs,https://maps.google.com/?cid=42524471774536901...,Cơm niêu KOMBO - L5-08 TTTM Parcmall,OPERATIONAL,"L5 08 TTTM Parcmall, Phường 15",3.7,Vừa,4.0,2024-09-01,"Phần cơm trung bình 90k/phần, nước trung bình ...",...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None
1,ChIJp1UGz2YpdTERL109bi26Azs,https://maps.google.com/?cid=42524471774536901...,Cơm niêu KOMBO - L5-08 TTTM Parcmall,OPERATIONAL,"L5 08 TTTM Parcmall, Phường 15",3.7,Vừa,5.0,2024-09-01,"Cơm lạ miệng, có lớp cơm cháy giòn giòn ăn ngo...",...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None
2,ChIJTYxwqQQpdTERu-toZpUuJhY,https://maps.google.com/?cid=15960143371525764...,Trịnh trần phương tuấn,OPERATIONAL,"RJFH+6VM, Phường 15",4.2,None,5.0,2019-09-01,Tuyệt vời,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None
3,ChIJNd_JkGIpdTERXRtR4acBq-w,https://maps.google.com/?cid=17053726234519870...,Domino's Pizza Đường Phan Huy Ích,OPERATIONAL,"7 Phan Huy Ích, Phường 15",4.4,None,5.0,2024-11-01,🌟🌟🌟🌟🌟\n\nMình rất hài lòng khi ghé Domino's Pi...,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None
4,ChIJNd_JkGIpdTERXRtR4acBq-w,https://maps.google.com/?cid=17053726234519870...,Domino's Pizza Đường Phan Huy Ích,OPERATIONAL,"7 Phan Huy Ích, Phường 15",4.4,None,4.0,2024-11-01,"Cỡ 9 có thể 2ng ăn ,sức ăn mạnh chắc 1 người ăn.",...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26787,ChIJ-ZE4DgApdTERS0vMWv7kdJQ,None,Sỏi Thái - Gò Vấp,OPERATIONAL,"Sỏi Thái - Gò Vấp, 246, Đ. Lê Đức Thọ, Gò Vấp",4.9,200-300,5.0,2024-11-01,"Quán mát, thoáng, Phục vụ tốt",...,None,None,None,None,None,None,5.0,5,5.0,NaN
26788,ChIJ-ZE4DgApdTERS0vMWv7kdJQ,None,Sỏi Thái - Gò Vấp,OPERATIONAL,"Sỏi Thái - Gò Vấp, 246, Đ. Lê Đức Thọ, Gò Vấp",4.9,200-300,5.0,2024-11-01,"Không gian rộng , thoáng mát , nv nhiệt tình",...,None,None,None,None,None,None,5.0,5,5.0,NaN
26789,ChIJZwyNWREpdTERLCUiK91rmSM,None,BeefSteak 100gram,OPERATIONAL,"BeefSteak 100gram, 221/5, Đ. Lê Đức Thọ, Gò Vấp",4.5,1-100.000,4.0,2024-11-01,"Đồ ăn ngon , giá phải chăng (2 ng 4 món hết 20...",...,None,None,None,None,None,None,4.0,5,5.0,NaN
26790,ChIJLdqfCwYpdTERHe9GR3p-jmg,None,Chay Mandala - Ẩm thực chay và Lẩu chay,OPERATIONAL,"Chay Mandala - Ẩm thực chay và Lẩu chay, 44K, ...",4.6,1-100.000,5.0,2024-11-01,"Lẩu thái chay rất ngon, sạch sẽ, có nhiều rau ...",...,None,None,None,None,None,None,5.0,5,5.0,Bữa tối


In [84]:
from IPython.display import display, clear_output

location_mapping = {}
for location in df[df['link'].isnull()].location.unique():
  display(location)
  location_mapping[location] = input("link :")

'Kichirou Ramen, 61, Phan Huy Ích, Tân Bình'

link :https://www.google.com/maps/place/Kichirou+Ramen/@10.8279186,106.6197933,1086m/data=!3m1!1e3!4m6!3m5!1s0x31752900012bd313:0x86bfd43015891aed!8m2!3d10.8272328!4d106.6314772!16s%2Fg%2F11ldhbvyyw?entry=ttu&g_ep=EgoyMDI1MDExNS4wIKXMDSoASAFQAw%3D%3D


'Panda BBQ Phan Văn Hớn, 56, Phan Văn Hớn, Quận 12'

link :https://www.google.com/maps/place/Panda+BBQ+Phan+V%C4%83n+H%E1%BB%9Bn/@10.8279186,106.6197938,1086m/data=!3m2!1e3!4b1!4m6!3m5!1s0x31752b9329074fb5:0x20ecc63ba8d12a80!8m2!3d10.8279133!4d106.6223682!16s%2Fg%2F11thc19758?entry=ttu&g_ep=EgoyMDI1MDExNS4wIKXMDSoASAFQAw%3D%3D


'Sushi Family, 62, Dương Thị Giang, Quận 12'

...

link :https://www.google.com/maps/place/BeefSteak+100gram/@10.8389986,106.6769098,1086m/data=!3m2!1e3!4b1!4m6!3m5!1s0x31752911598d0c67:0x23996bdd2b22252c!8m2!3d10.8389933!4d106.6794847!16s%2Fg%2F11s7czjzdf?entry=ttu&g_ep=EgoyMDI1MDExNS4wIKXMDSoASAFQAw%3D%3D


'Chay Mandala - Ẩm thực chay và Lẩu chay, 44K, Đ. Nguyễn Thị Đặng, Quận 12'

link :https://www.google.com/maps/place/Chay+Mandala+-+%E1%BA%A8m+th%E1%BB%B1c+chay+v%C3%A0+L%E1%BA%A9u+chay/@10.8741048,106.6427086,1086m/data=!3m2!1e3!4b1!4m6!3m5!1s0x317529060b9fda2d:0x688e7e7a4746ef1d!8m2!3d10.8740995!4d106.6452835!16s%2Fg%2F11mqwj93q5?entry=ttu&g_ep=EgoyMDI1MDExNS4wIKXMDSoASAFQAw%3D%3D


'KFC Nguyễn Ảnh Thủ 2, 787, Đ. Nguyễn Ảnh Thủ, Quận 12'

link :https://www.google.com/maps/place/KFC+Nguy%E1%BB%85n+%E1%BA%A2nh+Th%E1%BB%A7+2/@10.8752142,106.6206459,1086m/data=!3m2!1e3!4b1!4m6!3m5!1s0x31752baa777cdde9:0xa20c43cf548305c1!8m2!3d10.8752089!4d106.6232208!16s%2Fg%2F11k0n3fb8f?entry=ttu&g_ep=EgoyMDI1MDExNS4wIKXMDSoASAFQAw%3D%3D


In [86]:
df

,place_id,link,name,status,location,overall_rating,price,rating,review_time,comment,...,subpremise,sublocality_level_1,premise,administrative_area_level_3,point_of_interest,locality,Đồ ăn,Dịch vụ,Bầu không khí,Loại hình bữa ăn
0,ChIJp1UGz2YpdTERL109bi26Azs,https://maps.google.com/?cid=42524471774536901...,Cơm niêu KOMBO - L5-08 TTTM Parcmall,OPERATIONAL,"L5 08 TTTM Parcmall, Phường 15",3.7,Vừa,4.0,2024-09-01,"Phần cơm trung bình 90k/phần, nước trung bình ...",...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None
1,ChIJp1UGz2YpdTERL109bi26Azs,https://maps.google.com/?cid=42524471774536901...,Cơm niêu KOMBO - L5-08 TTTM Parcmall,OPERATIONAL,"L5 08 TTTM Parcmall, Phường 15",3.7,Vừa,5.0,2024-09-01,"Cơm lạ miệng, có lớp cơm cháy giòn giòn ăn ngo...",...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None
2,ChIJTYxwqQQpdTERu-toZpUuJhY,https://maps.google.com/?cid=15960143371525764...,Trịnh trần phương tuấn,OPERATIONAL,"RJFH+6VM, Phường 15",4.2,None,5.0,2019-09-01,Tuyệt vời,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None
3,ChIJNd_JkGIpdTERXRtR4acBq-w,https://maps.google.com/?cid=17053726234519870...,Domino's Pizza Đường Phan Huy Ích,OPERATIONAL,"7 Phan Huy Ích, Phường 15",4.4,None,5.0,2024-11-01,🌟🌟🌟🌟🌟\n\nMình rất hài lòng khi ghé Domino's Pi...,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None
4,ChIJNd_JkGIpdTERXRtR4acBq-w,https://maps.google.com/?cid=17053726234519870...,Domino's Pizza Đường Phan Huy Ích,OPERATIONAL,"7 Phan Huy Ích, Phường 15",4.4,None,4.0,2024-11-01,"Cỡ 9 có thể 2ng ăn ,sức ăn mạnh chắc 1 người ăn.",...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26787,ChIJ-ZE4DgApdTERS0vMWv7kdJQ,https://www.google.com/maps/place/S%E1%BB%8Fi+...,Sỏi Thái - Gò Vấp,OPERATIONAL,"Sỏi Thái - Gò Vấp, 246, Đ. Lê Đức Thọ, Gò Vấp",4.9,200-300,5.0,2024-11-01,"Quán mát, thoáng, Phục vụ tốt",...,None,None,None,None,None,None,5.0,5,5.0,NaN
26788,ChIJ-ZE4DgApdTERS0vMWv7kdJQ,https://www.google.com/maps/place/S%E1%BB%8Fi+...,Sỏi Thái - Gò Vấp,OPERATIONAL,"Sỏi Thái - Gò Vấp, 246, Đ. Lê Đức Thọ, Gò Vấp",4.9,200-300,5.0,2024-11-01,"Không gian rộng , thoáng mát , nv nhiệt tình",...,None,None,None,None,None,None,5.0,5,5.0,NaN
26789,ChIJZwyNWREpdTERLCUiK91rmSM,https://www.google.com/maps/place/BeefSteak+10...,BeefSteak 100gram,OPERATIONAL,"BeefSteak 100gram, 221/5, Đ. Lê Đức Thọ, Gò Vấp",4.5,1-100.000,4.0,2024-11-01,"Đồ ăn ngon , giá phải chăng (2 ng 4 món hết 20...",...,None,None,None,None,None,None,4.0,5,5.0,NaN
26790,ChIJLdqfCwYpdTERHe9GR3p-jmg,https://www.google.com/maps/place/Chay+Mandala...,Chay Mandala - Ẩm thực chay và Lẩu chay,OPERATIONAL,"Chay Mandala - Ẩm thực chay và Lẩu chay, 44K, ...",4.6,1-100.000,5.0,2024-11-01,"Lẩu thái chay rất ngon, sạch sẽ, có nhiều rau ...",...,None,None,None,None,None,None,5.0,5,5.0,Bữa tối


In [87]:
df.loc[0]

,0
place_id,ChIJp1UGz2YpdTERL109bi26Azs
link,https://maps.google.com/?cid=42524471774536901...
name,Cơm niêu KOMBO - L5-08 TTTM Parcmall
status,OPERATIONAL
location,"L5 08 TTTM Parcmall, Phường 15"
overall_rating,3.7
price,Vừa
rating,4.0
review_time,2024-09-01 00:00:00
comment,"Phần cơm trung bình 90k/phần, nước trung bình ..."


In [88]:
df.to_csv(f"{data_path}/filtered_data.csv", index=False)